# Agents as Tools with Strands Agents


"Agents as Tools" is an architectural pattern in AI systems where specialized AI agents are wrapped as callable functions (tools) that can be used by other agents. This creates a hierarchical structure where:

1. A primary "orchestrator" agent handles user interaction and determines which specialized agent to call

2. Specialized "tool agents" perform domain-specific tasks when called by the orchestrator

This approach mimics human team dynamics, where a manager coordinates specialists, each bringing unique expertise to solve complex problems. Rather than a single agent trying to handle everything, tasks are delegated to the most appropriate specialized agent.



## Key Benefits and Core Principles

The "Agents as Tools" pattern offers several advantages:

- Separation of concerns: Each agent has a focused area of responsibility, making the system easier to understand and maintain
- Hierarchical delegation: The orchestrator decides which specialist to invoke, creating a clear chain of command
- Modular architecture: Specialists can be added, removed, or modified independently without affecting the entire system
- Improved performance: Each agent can have tailored system prompts and tools optimized for its specific task


In [1]:
!pip install -r requirements.txt

In [2]:
import os

from strands import Agent, tool
from strands_tools import file_write

In this module we will be creating an orchestrator based multi-agent workflow. 

<div style="text-align:left">
    <img src="images/architecture.png" width="75%" />
</div>

We will also explore `use_llm` which allows use to create nested agents.

## Research Agent

Lets first create a basic reasearch assistant with http_request tool. 

In [3]:
RESEARCH_ASSISTANT_PROMPT = """You are a specialized research assistant. Focus only on providing
factual, well-sourced information in response to research questions.
Always cite your sources when possible."""

In [4]:
research_agent = Agent(
    system_prompt=RESEARCH_ASSISTANT_PROMPT,
    # tools=[http_request]  # Here you can enable an agentic ai search tool
)

query = "Overview of Amazon Bedrock and its features"
# Call the agent and return its response
response = research_agent(query)

# Amazon Bedrock Overview

Amazon Bedrock is a fully managed service launched by AWS in September 2023 that provides access to foundation models (FMs) from leading AI companies through a single API.

## Core Features

### **Foundation Model Access**
- **Multiple Providers**: Access to models from Anthropic (Claude), AI21 Labs (Jurassic), Cohere (Command/Embed), Meta (Llama 2), Stability AI (Stable Diffusion), and Amazon's own Titan models
- **Model Variety**: Supports text generation, chat, text summarization, image generation, and embeddings
- **Unified API**: Single interface to interact with different foundation models without managing separate integrations

### **Customization Capabilities**
- **Fine-tuning**: Customize models with your own training data while keeping data private
- **Retrieval Augmented Generation (RAG)**: Connect models to your data sources for more accurate, contextual responses
- **Prompt Engineering**: Built-in tools for optimizing prompts and model responses


Now we can wrap this agent as a tool. Allowing other agents to interact with it. 

####  Best Practices for Agent as Tools

When implementing the "Agents as Tools" pattern with Strandly AI:

1. Clear tool documentation: Write descriptive docstrings that explain the agent's expertise
2. Focused system prompts: Keep each specialized agent tightly focused on its domain
3. Proper response handling: Use consistent patterns to extract and format responses
4. Tool selection guidance: Give the orchestrator clear criteria for when to use each specialized agen

In [5]:
@tool
def research_assistant(query: str) -> str:
    """
    Process and respond to research-related queries.

    Args:
        query: A research question requiring factual information

    Returns:
        A detailed research answer with citations
    """
    try:
        # Strands agents makes it easy to create a specialized agent
        research_agent = Agent(
            system_prompt=RESEARCH_ASSISTANT_PROMPT,
        )

        # Call the agent and return its response
        response = research_agent(query)
        return str(response)
    except Exception as e:
        return f"Error in research assistant: {str(e)}"

Now lets follow the best practices and create `product_recommendation_assistant`, `trip_planning_assistant`, and `orchestrator` agent.

### Product Recommendation Assistant

In [6]:
@tool
def product_recommendation_assistant(query: str) -> str:
    """
    Handle product recommendation queries by suggesting appropriate products.

    Args:
        query: A product inquiry with user preferences

    Returns:
        Personalized product recommendations with reasoning
    """
    try:
        product_agent = Agent(
            system_prompt="""You are a specialized product recommendation assistant.
            Provide personalized product suggestions based on user preferences. Always cite your sources.""",
        )
        # Call the agent and return its response
        response = product_agent(query)

        return str(response)
    except Exception as e:
        return f"Error in product recommendation: {str(e)}"

In [7]:
product_recommendation_assistant("Product recommendations for flying cars")

I should clarify that commercially available flying cars for consumers don't currently exist in the traditional sense. However, I can share information about emerging products and concepts in this space:

## Current Development Stage Products:

**1. eVTOL Aircraft (Electric Vertical Takeoff and Landing)**
- **Joby Aviation S4**: In development for air taxi services
- **Lilium Jet**: 5-seater electric aircraft for regional travel
- **EHang EH216-S**: Autonomous aerial vehicle for short trips

**2. Roadable Aircraft (Street-Legal Flying Vehicles)**
- **Terrafugia Transition**: FAA-approved light sport aircraft that can drive on roads
- **PAL-V Liberty**: Dutch-made gyrocopter/car hybrid (limited production)
- **AeroMobil**: Slovakian flying car in development

## Important Considerations:
- These vehicles require pilot licenses and are subject to aviation regulations
- Prices typically range from $300,000 to $2+ million
- Most are still in testing/pre-production phases
- Infrastructure f

"I should clarify that commercially available flying cars for consumers don't currently exist in the traditional sense. However, I can share information about emerging products and concepts in this space:\n\n## Current Development Stage Products:\n\n**1. eVTOL Aircraft (Electric Vertical Takeoff and Landing)**\n- **Joby Aviation S4**: In development for air taxi services\n- **Lilium Jet**: 5-seater electric aircraft for regional travel\n- **EHang EH216-S**: Autonomous aerial vehicle for short trips\n\n**2. Roadable Aircraft (Street-Legal Flying Vehicles)**\n- **Terrafugia Transition**: FAA-approved light sport aircraft that can drive on roads\n- **PAL-V Liberty**: Dutch-made gyrocopter/car hybrid (limited production)\n- **AeroMobil**: Slovakian flying car in development\n\n## Important Considerations:\n- These vehicles require pilot licenses and are subject to aviation regulations\n- Prices typically range from $300,000 to $2+ million\n- Most are still in testing/pre-production phases\

### Trip Planning Assistant

In [8]:
@tool
def trip_planning_assistant(query: str) -> str:
    """
    Create travel itineraries and provide travel advice.

    Args:
        query: A travel planning request with destination and preferences

    Returns:
        A detailed travel itinerary or travel advice
    """
    try:
        travel_agent = Agent(
            system_prompt="""You are a specialized travel planning assistant.
            Create detailed travel itineraries based on user preferences.""",
        )
        # Call the agent and return its response
        response = travel_agent(query)

        return str(response)
    except Exception as e:
        return f"Error in trip planning: {str(e)}"

### Orchestrator Agent

In [9]:
# Define orchestrator system prompt with clear tool selection guidance
MAIN_SYSTEM_PROMPT = """
You are an assistant that routes queries to specialized agents:
- For research questions and factual information → Use the research_assistant tool
- For product recommendations and shopping advice → Use the product_recommendation_assistant tool
- For travel planning and itineraries → Use the trip_planning_assistant tool
- For simple questions not requiring specialized knowledge → Answer directly

Always select the most appropriate tool based on the user's query.
"""

In [10]:
# Strands Agents allows easy integration of agent tools
orchestrator = Agent(
    system_prompt=MAIN_SYSTEM_PROMPT,
    tools=[
        research_assistant,
        product_recommendation_assistant,
        trip_planning_assistant,
        file_write,
    ],
)

In [12]:
!pip install langfuse

In [ ]:
from langfuse import Langfuse

# Get keys for your project from the project settings page: https://cloud.langfuse.com
public_key = "pk"
secret_key = "sk"

os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com" # 🇪🇺 EU region
#os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com" # 🇺🇸 US region

# Set up endpoint
otel_endpoint = str(os.environ.get("LANGFUSE_HOST")) + "/api/public/otel/v1/traces"

# Create authentication token:
import base64
auth_token = base64.b64encode(f"{public_key}:{secret_key}".encode()).decode()
os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = otel_endpoint
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {auth_token}"

langfuse = Langfuse(
    public_key=public_key,
    secret_key=secret_key,
    host="https://cloud.langfuse.com"
)

In [14]:
# Example: E-commerce Customer Service System
customer_query = (
    "I'm looking for hiking boots. Write the final response to current directory."
)

os.environ["BYPASS_TOOL_CONSENT"] = "true"

# The orchestrator automatically determines this requires multiple specialized agents
response = orchestrator(customer_query)

I'll help you find hiking boots and save the response to the current directory.
Tool #3: product_recommendation_assistant
I'd be happy to help you find the right hiking boots! To give you the best recommendations, could you tell me more about:

1. **Your hiking style**: Day hikes, backpacking, or technical terrain?
2. **Budget range**: What are you comfortable spending?
3. **Foot characteristics**: Any specific fit needs (wide feet, arch support, etc.)?
4. **Climate/terrain**: Where will you primarily be hiking?

That said, here are some widely recommended categories to consider:

**For Day Hiking:**
- Merrell Moab series (versatile, comfortable)
- Salomon X Ultra series (lightweight, good traction)

**For Backpacking:**
- Lowa Renegade GTX (durable, waterproof)
- Keen Targhee series (roomy toe box, sturdy)

**For Technical Terrain:**
- La Sportiva boots (precise fit, climbing capability)
- Scarpa models (Italian craftsmanship, technical features)

**Key features to look for:**
- Prope

╔══════ File Write Operation ═══════╗
║                                   ║
║ Path: ./hiking_boots_response.txt ║
║ Size: 1293 characters             ║
║                                   ║
╚═══════════════════════════════════╝

╔════════════════════ Write Successful ════════════════════╗
║ File written successfully to ./hiking_boots_response.txt ║
╚══════════════════════════════════════════════════════════╝

I've provided hiking boot recommendations and saved the complete response to a file called "hiking_boots_response.txt" in your current directory. The response includes recommendations for different hiking styles (day hiking, backpacking, technical terrain), specific boot brands like Merrell, Salomon, Lowa, and others, plus key features to consider when selecting boots.

The file contains guidance on what information would help narrow down more specific recommendations, such as your hiking style, budget, foot characteristics, and terrain preferences.

Lets look at the messages of the orchestrator. Here you can see the agent decided to use the sub-agent as tool

In [15]:
orchestrator.messages

[{'role': 'user',
  'content': [{'text': "I'm looking for hiking boots. Write the final response to current directory."}]},
 {'role': 'assistant',
  'content': [{'text': "I'll help you find hiking boots and then save the response to a file in the current directory."},
   {'toolUse': {'toolUseId': 'tooluse_ocD1raI3Rm6EF9HWmmI0sg',
     'name': 'product_recommendation_assistant',
     'input': {'query': "I'm looking for hiking boots"}}}]},
 {'role': 'user',
  'content': [{'toolResult': {'toolUseId': 'tooluse_ocD1raI3Rm6EF9HWmmI0sg',
     'status': 'success',
     'content': [{'text': "I'd be happy to help you find the right hiking boots! To give you the best recommendations, could you tell me more about:\n\n**Your hiking style:**\n- Day hikes, backpacking, or casual trails?\n- What terrain (rocky, muddy, desert, etc.)?\n\n**Your preferences:**\n- Budget range?\n- Foot width (narrow, regular, wide)?\n- Any specific features needed (waterproof, lightweight, ankle support)?\n\n**General cat

In [16]:
customer_query = "Can you help me plan my trip to Patagonia"

response = orchestrator(customer_query)

I'd be happy to help you plan your trip to Patagonia! Let me get you a detailed travel plan.
Tool #5: trip_planning_assistant
I'd love to help you plan an amazing trip to Patagonia! This is one of the world's most spectacular wilderness destinations. To create the best itinerary for you, I need to understand your preferences better.

Could you please share:

**Essential Details:**
- **Duration**: How many days/weeks do you have?
- **Travel dates**: When are you planning to visit?
- **Budget range**: What's your approximate budget?
- **Fitness level**: Are you comfortable with strenuous hiking?

**Preferences:**
- **Activities**: Hiking, mountaineering, wildlife viewing, photography, cultural experiences?
- **Accommodation style**: Camping, hostels, hotels, luxury lodges?
- **Side preference**: Chilean Patagonia, Argentine Patagonia, or both?

**Key Patagonia Highlights to Consider:**
- **Torres del Paine** (Chile) - Iconic granite towers, world-class trekking
- **Los Glaciares National

In [17]:
orchestrator.messages

[{'role': 'user',
  'content': [{'text': "I'm looking for hiking boots. Write the final response to current directory."}]},
 {'role': 'assistant',
  'content': [{'text': "I'll help you find hiking boots and then save the response to a file in the current directory."},
   {'toolUse': {'toolUseId': 'tooluse_ocD1raI3Rm6EF9HWmmI0sg',
     'name': 'product_recommendation_assistant',
     'input': {'query': "I'm looking for hiking boots"}}}]},
 {'role': 'user',
  'content': [{'toolResult': {'toolUseId': 'tooluse_ocD1raI3Rm6EF9HWmmI0sg',
     'status': 'success',
     'content': [{'text': "I'd be happy to help you find the right hiking boots! To give you the best recommendations, could you tell me more about:\n\n**Your hiking style:**\n- Day hikes, backpacking, or casual trails?\n- What terrain (rocky, muddy, desert, etc.)?\n\n**Your preferences:**\n- Budget range?\n- Foot width (narrow, regular, wide)?\n- Any specific features needed (waterproof, lightweight, ankle support)?\n\n**General cat

### Calling multiple agents 

In [18]:
orchestrator.messages = []

In [ ]:
query = "Can you do a research on spain? Also help me plan a 7 day trip."

orchestrator(query)

Behind the scenes, the orchestrator will:
1. First call the `research_assistant`
2. Then call `trip_planning_assistant`
3. Combine these specialized responses into a cohesive answer that addresses both the queries

### Sequential Agent Communication Pattern


The agent tool can also combine multiple agents together. In this example we will provide output of `research_agent` to `summary_agent` and return the summarized response.

In [ ]:
 # define the user query
topic = "generative Ai"
# Create a research agent
research_agent = Agent(
    system_prompt=RESEARCH_ASSISTANT_PROMPT,
)
# Create a summarization agent
summary_agent = Agent(
    system_prompt="""
    You are a summarization specialist focused on distilling complex information into clear, concise summaries.
    Your primary goal is to take detailed information and extract the key points, main arguments, and critical data.
    You should maintain the accuracy of the original content while making it more digestible.
    Focus on clarity, brevity, and highlighting the most important aspects of the information.
    """,
)

print("Multiple agents created successfully!")
print(f"\n🔍 RESEARCH AGENT working on: {topic}\n") 
try:
    # Agent 1: Invoke research agent
    research_response = research_agent(
        f"Please gather comprehensive information about {topic}."
    )
    research_text = research_response.message.content[0]["text"]
    print("\n✂️ SUMMARY AGENT distilling the research\n")
    
    # Agent 2: Ask the summary agent to create a concise summary
    summary_response = summary_agent(
        f"Please create a concise summary of this research: {research_text}"
    )
    summary_text = summary_response.message.content[0]["text"]
    
    print(summary_text)
except Exception as e:
    print(f"Error in research assistant: {str(e)}")

## Congrats!

You've learned how to use agents as tools in Strands Agents to create more complex agentic applications